In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Jan 11 21:24:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    44W / 350W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')
#!ln -s /content/gdrive/My\ Drive/ /mydrive
#!ls /mydrive

####1) Clone the Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.15 MiB | 18.72 MiB/s, done.
Resolving deltas: 100% (10404/10404), done.


####2) Compile Darknet using Nvidia GPU

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

####3) Configure Darknet network for training YOLO V3

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-01-11 21:26:35--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  18.5MB/s    in 9.5s    

2023-01-11 21:26:45 (16.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



#### 4) Extract Images

In [ ]:
#The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/GHI.txt        
  inflating: data/obj/01.jpg         
  inflating: data/obj/01.txt         
  inflating: data/obj/1_2_3.jpg      
  inflating: data/obj/1_2_3.txt      
  inflating: data/obj/02.jpg         
  inflating: data/obj/02.txt         
  inflating: data/obj/2_3_1.jpg      
  inflating: data/obj/2_3_1.txt      
  inflating: data/obj/03.jpg         
  inflating: data/obj/03.txt         
  inflating: data/obj/3_7_0.jpg      
  inflating: data/obj/3_7_0.txt      
  inflating: data/obj/04.jpg         
  inflating: data/obj/04.txt         
  inflating: data/obj/4_1_5.jpg      
  inflating: data/obj/4_1_5.txt      
  inflating: data/obj/05.jpg         
  inflating: data/obj/05.txt         
  inflating: data/obj/5_3_1.jpg      
  inflating: data/obj/5_3_1.txt      
  inflating: data/obj/06.jpg         
  inflating: data/obj/06.txt         
  inflating: data/obj/6_7_0.jpg      
  inflating: data/obj/6_7_0.txt      
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

43 0.5540685224839401 0.29710920770877947 0.2826552462526767 0.4421841541755889
['43', '0.5540685224839401', '0.29710920770877947', '0.2826552462526767', '0.4421841541755889']
0 data/obj/ABC.txt
0 0.5540685224839401 0.29710920770877947 0.2826552462526767 0.4421841541755889
33 0.5085653104925054 0.3308351177730193 0.202355460385439 0.5674518201284797
['33', '0.5085653104925054', '0.3308351177730193', '0.202355460385439', '0.5674518201284797']
1 data/obj/65.txt
0 0.5085653104925054 0.3308351177730193 0.202355460385439 0.5674518201284797
65 0.37152034261241973 0.6429336188436832 0.29229122055674517 0.40364025695931494
['65', '0.37152034261241973', '0.6429336188436832', '0.29229122055674517', '0.40364025695931494']
2 data/obj/39.txt
0 0.37152034261241973 0.6429336188436832 0.29229122055674517 0.40364025695931494
116 0.5048179871520343 0.5305139186295503 0.31477516059957167 0.9389721627408993
['116', '0.5048179871520343', '0.5305139186295503', '0.31477516059957167', '0.9389721627408993']
3 

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/64_14.jpg', 'data/obj/9_2_2.jpg', 'data/obj/06.jpg', 'data/obj/51.jpg', 'data/obj/256_10.jpg', 'data/obj/13.jpg', 'data/obj/64_09.jpg', 'data/obj/24.jpg', 'data/obj/256_06.jpg', 'data/obj/46.jpg', 'data/obj/64_04.jpg', 'data/obj/64_10.jpg', 'data/obj/09.jpg', 'data/obj/45.jpg', 'data/obj/19_0_3.jpg', 'data/obj/36.jpg', 'data/obj/64_03.jpg', 'data/obj/17_6_2.jpg', 'data/obj/1_2_3.jpg', 'data/obj/34.jpg', 'data/obj/53.jpg', 'data/obj/256_08.jpg', 'data/obj/256_18.jpg', 'data/obj/ABC.jpg', 'data/obj/22.jpg', 'data/obj/4_1_5.jpg', 'data/obj/14.jpg', 'data/obj/38.jpg', 'data/obj/15.jpg', 'data/obj/64_15.jpg', 'data/obj/64_01.jpg', 'data/obj/30.jpg', 'data/obj/35.jpg', 'data/obj/256_16.jpg', 'data/obj/7_6_2.jpg', 'data/obj/6_7_0.jpg', 'data/obj/70.jpg', 'data/obj/47.jpg', 'data/obj/16.jpg', 'data/obj/52.jpg', 'data/obj/69.jpg', 'data/obj/68.jpg', 'data/obj/48.jpg', 'data/obj/03.jpg', 'data/obj/19.jpg', 'data/obj/71.jpg', 'data/obj/44.jpg', 'data/obj/256_13.jpg', 'data/obj/39.jpg',

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

####6) Start the training

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

A saída de streaming foi truncada nas últimas 5000 linhas.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.831544), count: 1, class_loss = 0.000391, iou_loss = 0.018853, total_loss = 0.019243 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 237656, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.892026), count: 4, class_loss = 0.013811, iou_loss = 0.064211, total_loss = 0.078022 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000006, iou_loss = 0.000000, total_loss = 0.000006 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 237660, rewritten_bbox = 0.000000 